In [1]:
import pandas as pd

from pitch_geo.infer_utils import scale_xy

In [8]:
! ls prediction_on_validation_data_*.csv -l

-rw-r--r-- 1 tensorflow-user tensorflow-user 1531424 Dec 19 23:50 prediction_on_validation_data_20221124_094146.csv
-rw-r--r-- 1 tensorflow-user tensorflow-user 1592814 Dec 19 23:43 prediction_on_validation_data_20221218_235616.csv
-rw-r--r-- 1 tensorflow-user tensorflow-user 1592528 Dec 20 10:51 prediction_on_validation_data_20221220_102058.csv
-rw-r--r-- 1 tensorflow-user tensorflow-user 1592961 Dec 20 11:59 prediction_on_validation_data_20221220_105705.csv
-rw-r--r-- 1 tensorflow-user tensorflow-user 1592621 Dec 20 12:47 prediction_on_validation_data_20221220_120024.csv


In [2]:
# Results on the validation set from the model 20221124_094146
df1  = pd.read_csv('prediction_on_validation_data_20221124_094146.csv', index_col=0)

# Results on the validation set from the model 20221218_235616
df2  = pd.read_csv('prediction_on_validation_data_20221218_235616.csv', index_col=0)

df3  = pd.read_csv('prediction_on_validation_data_20221220_102058.csv', index_col=0)

df4  = pd.read_csv('prediction_on_validation_data_20221220_105705.csv', index_col=0)

df5  = pd.read_csv('prediction_on_validation_data_20221220_120024.csv', index_col=0)

df6  = pd.read_csv('prediction_on_validation_data_20221220_153616.csv', index_col=0)

# True results
df_true = pd.read_csv('true_validation_data.csv', index_col=0)
df_true = scale_xy(df_true)

In [24]:
all(df1['image_path'].unique() == df2['image_path'].unique())

True

In [25]:
all(df1['image_path'].unique() == df_true['image_path'].unique())

True

In [3]:
df1 = df1[df1['kid'].isin(df_true['kid'].unique())]
df2 = df2[df2['kid'].isin(df_true['kid'].unique())]
df3 = df3[df3['kid'].isin(df_true['kid'].unique())]
df4 = df4[df4['kid'].isin(df_true['kid'].unique())]
df5 = df5[df5['kid'].isin(df_true['kid'].unique())]
df6 = df6[df6['kid'].isin(df_true['kid'].unique())]

In [4]:
print(len(df1), len(df3), len(df_true))

19516 19516 19516


In [4]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)
df3.reset_index(inplace=True)
df4.reset_index(inplace=True)
df5.reset_index(inplace=True)
df6.reset_index(inplace=True)
df_true.reset_index(inplace=True)

In [5]:
def corrdinates_loss(true, pred):
    mask = true['vis'] != 0
    
    x = (true.loc[mask]['x'] - pred.loc[mask]['x']) ** 2
    y = (true.loc[mask]['y'] - pred.loc[mask]['y']) ** 2
    
    loss = (x + y).transform('sqrt').agg('mean')
    return loss

In [6]:
corrdinates_loss(df_true, df1)

141.66641324737935

In [45]:
corrdinates_loss(df_true, df2)

133.84014774427985

It has almost the same recall and precision as the model 20221124_094146, but coordinates loss is twice as low!


|          | 20221124_094146 | 20221220_102058 |
|----------|-----------------|-----------------|
|precision | 0.971           | 0.9686          |
|recall    | 0.9708          | 0.9688          |

In [7]:
# weight = .5 or 0.9 ???
corrdinates_loss(df_true, df3)

74.4954285031414

In [24]:
# weight = 1.0
corrdinates_loss(df_true, df4)

67.79878906843044

This model has even lower coordinates loss, but its precision and recall are a bit poorer

|          | 20221124_094146 | 20221220_102058 |
|----------|-----------------|-----------------|
|precision | 0.971           | 0.9529          |
|recall    | 0.9708          | 0.9296          |

In [25]:
# weight = .999
corrdinates_loss(df_true, df5)

65.08887638576492

|          | 20221124_094146 | 20221220_153616 |
|----------|-----------------|-----------------|
|precision | 0.971           | 0.9661          |
|recall    | 0.9708          | 0.9693          |

In [33]:
# weight = .95
corrdinates_loss(df_true, df6)

71.39714513855259

In [40]:
test_df = pd.read_csv('test_keypoints_teacher_forcing.csv')

In [41]:
test_df

,x,y,vis,dataset,kid,image_path
0,0,0,0,test,1,images/test/0117b5a019bafe1bff3b93516fc1be.jpg
1,258,274,2,test,2,images/test/0117b5a019bafe1bff3b93516fc1be.jpg
2,0,0,0,test,3,images/test/0117b5a019bafe1bff3b93516fc1be.jpg
3,0,0,0,test,4,images/test/0117b5a019bafe1bff3b93516fc1be.jpg
4,0,0,0,test,5,images/test/0117b5a019bafe1bff3b93516fc1be.jpg
...,...,...,...,...,...,...
11305,0,0,0,test,35,images/test/fe0ae584ba268e6292f553d7494f27.jpg
11306,0,0,0,test,36,images/test/fe0ae584ba268e6292f553d7494f27.jpg
11307,0,0,0,test,37,images/test/fe0ae584ba268e6292f553d7494f27.jpg
11308,0,0,0,test,38,images/test/fe0ae584ba268e6292f553d7494f27.jpg
